# Ch 5

In [1]:
# CNN. ConvNet

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\Users\MBIS\anaconda3\envs\tensorflow114_py37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\MBIS\anaconda3\envs\tensorflow114_py37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\MBIS\anaconda3\envs\tensorflow114_py37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\MBIS\anaconda3\envs\tensorfl

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
import tensorflow as tf

x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

In [4]:
x_image  = tf.reshape(x,[-1,28,28,1])

In [5]:
# initialize weight => break symmetric and 0 gradient, give noise..! (unstable)
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

# + give small positive value '0.1'
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [6]:
# convolution
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# padding: output size == input size


# pooling
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [7]:
# first convolution layer (convolition + max pooling)
# W_conv1 : 5x5 patch(window) 1 input Ch, 32 filters.
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# input image (x => reshape 4D tensor. -1..? , width, height, # of Color CH)
# => 4D tensor data
x_image = tf.reshape(x, [-1,28,28,1]) # 28x28

# apply convolution (relu(conv + bias)) + max pooling
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 14x14
# second convolution layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 7x7

# Fully connected layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


In [8]:
# Dropout : Prevent overfitting
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
# Final Softmax layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [10]:
# train and evaluation
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
# cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1])
# tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float32"))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(20000):
  batch = mnist.train.next_batch(100)
  if i%100 == 0:
    train_accuracy = sess.run(accuracy, feed_dict = {x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))            # print step and accuracy at every 100 step.
  sess.run(accuracy, feed_dict = {x:batch[0], y_: batch[1], keep_prob: 0.5})
# print("test accuracy %g"% sess.run(accuracy, sess.run(accuracy, feed_dict = {x:mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})))
                                       


step 0, training accuracy 0.13
step 100, training accuracy 0.14
step 200, training accuracy 0.17
step 300, training accuracy 0.12
step 400, training accuracy 0.13
step 500, training accuracy 0.11
step 600, training accuracy 0.17
step 700, training accuracy 0.18
step 800, training accuracy 0.14
step 900, training accuracy 0.11
step 1000, training accuracy 0.13
step 1100, training accuracy 0.08
step 1200, training accuracy 0.16
step 1300, training accuracy 0.1
step 1400, training accuracy 0.11
step 1500, training accuracy 0.2
step 1600, training accuracy 0.11
step 1700, training accuracy 0.13
step 1800, training accuracy 0.12
step 1900, training accuracy 0.17
step 2000, training accuracy 0.13
step 2100, training accuracy 0.16
step 2200, training accuracy 0.14
step 2300, training accuracy 0.17
step 2400, training accuracy 0.15
step 2500, training accuracy 0.12
step 2600, training accuracy 0.14
step 2700, training accuracy 0.13
step 2800, training accuracy 0.11
step 2900, training accuracy

TypeError: input must be a dictionary

In [11]:
# 해당 결과 확인에 에러 발생. (다른 예제에서도 문제 발생)
print("test accuracy %g"% sess.run(accuracy, sess.run(accuracy, feed_dict = {x:batch[0], y_: batch[1], keep_prob: 1.0})))


TypeError: input must be a dictionary

In [12]:
batch[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)